# ตัวอย่างตัวแทนจองโรงแรมและเที่ยวบิน

โซลูชันนี้จะช่วยคุณจองตั๋วเครื่องบินและโรงแรม สถานการณ์คือการเดินทางจากลอนดอน ฮีทโธรว์ (LHR) วันที่ 20 กุมภาพันธ์ 2024 ไปยังนิวยอร์ก (JFK) และกลับวันที่ 27 กุมภาพันธ์ 2025 โดยบินชั้นประหยัดกับสายการบินบริติชแอร์เวย์เท่านั้น ฉันต้องการพักที่โรงแรมฮิลตันในนิวยอร์ก กรุณาแจ้งค่าใช้จ่ายสำหรับเที่ยวบินและโรงแรม


# เริ่มต้นใช้งาน Azure AI Agent Service และรับข้อมูลการตั้งค่าจาก **.env**

### **.env** 

สร้างไฟล์ .env 

**.env** ประกอบด้วย connection string ของ Azure AI Agent Service, โมเดลที่ใช้โดย AOAI และ API ของ Google API Search service, ENDPOINT เป็นต้น

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "ชื่อการปรับใช้โมเดลของ Azure AI Agent Service ของคุณ"

[**NOTE**] คุณจะต้องมีโมเดลที่มี Rate Limit 100,000 (Tokens ต่อหนึ่งนาที) และ Rate Limit 600 (Request ต่อหนึ่งนาที)

  คุณสามารถรับโมเดลได้ใน Azure AI Foundry - Model และ Endpoint


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Connection String ของโปรเจกต์ Azure AI Agent Service ของคุณ"

  คุณสามารถรับ connection string ของโปรเจกต์ได้ในหน้าภาพรวมโปรเจกต์ใน AI ​​Foundry Portal Screen

- **SERPAPI_SEARCH_API_KEY** = "SERPAPI Search API KEY ของคุณ"
- **SERPAPI_SEARCH_ENDPOINT** = "SERPAPI Search Endpoint ของคุณ"

ในการรับชื่อการปรับใช้โมเดลและ connection string ของโปรเจกต์ Azure AI Agent Service คุณจำเป็นต้องสร้าง Azure AI Agent Service ขอแนะนำให้ใช้ [เทมเพลตนี้](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) เพื่อสร้างโดยตรง （***หมายเหตุ:*** Azure AI Agent Service ปัจจุบันถูกตั้งค่าในภูมิภาคที่จำกัด ขอแนะนำให้คุณอ้างอิง [ลิงก์นี้](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) เพื่อกำหนดภูมิภาค)

Agent จำเป็นต้องเข้าถึง SERPAPI ขอแนะนำให้ลงทะเบียนโดยใช้ [ลิงก์นี้](https://serpapi.com/searches) หลังจากลงทะเบียน คุณจะสามารถรับ API KEY และ ENDPOINT เฉพาะของคุณ


# การตั้งค่า

ในการใช้งานโน้ตบุ๊กนี้ คุณจำเป็นต้องตรวจสอบให้แน่ใจว่าคุณได้ติดตั้งไลบรารีที่จำเป็นแล้ว โดยรันคำสั่ง `pip install -r requirements.txt`


In [ ]:
from semantic_kernel import __version__

__version__

เวอร์ชัน Semantic Kernel ของคุณควรเป็นอย่างน้อย 1.27.2


โหลดการตั้งค่าและทรัพยากรจากไฟล์ .env ของคุณ กรุณาตรวจสอบให้แน่ใจว่าคุณได้เพิ่มคีย์และการตั้งค่า และสร้างไฟล์ .env ในเครื่องเรียบร้อยแล้ว


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# เข้าสู่ระบบ Azure

ตอนนี้คุณจำเป็นต้องเข้าสู่ระบบ Azure ให้เปิดเทอร์มินัลและรันคำสั่งต่อไปนี้:

```bash
az login
```

คำสั่งนี้จะขอให้คุณป้อนข้อมูลรับรอง Azure ของคุณ เพื่อให้บริการ Azure AI Agent ทำงานได้อย่างถูกต้อง


# คำอธิบาย:
นี่คือค่าตัวแปรที่ใช้เก็บ API key สำหรับการเข้าถึงบริการ SERP (Search Engine Results Page) API ซึ่ง API key เป็นตัวระบุเฉพาะที่ใช้ในการยืนยันคำขอที่เกี่ยวข้องกับบัญชีของคุณ

วัตถุประสงค์: วัตถุประสงค์ของบรรทัดนี้คือการเก็บ API key ไว้ในตัวแปรเพื่อใช้ในการยืนยันคำขอไปยังบริการ SERP API โดย API key เป็นสิ่งจำเป็นสำหรับการเข้าถึงบริการและการดำเนินการค้นหา
วิธีการรับ SERP API Key: เพื่อรับ SERP API key ให้ทำตามขั้นตอนทั่วไปที่ https://serpapi.com (ขั้นตอนที่แน่นอนอาจแตกต่างกันไปขึ้นอยู่กับบริการ SERP API ที่คุณใช้งาน):

เลือกบริการ SERP API: มีบริการ SERP API หลายตัวให้เลือก เช่น SerpAPI, Google Custom Search JSON API และอื่น ๆ เลือกบริการที่เหมาะสมกับความต้องการของคุณ

สมัครบัญชี: ไปที่เว็บไซต์ของบริการ SERP API ที่คุณเลือกและสมัครบัญชี คุณอาจต้องให้ข้อมูลพื้นฐานบางอย่างและยืนยันที่อยู่อีเมลของคุณ

สร้าง API Key: หลังจากสมัครบัญชีแล้ว ให้เข้าสู่ระบบและไปที่ส่วน API หรือแดชบอร์ด มองหาตัวเลือกในการสร้างหรือสร้าง API key ใหม่
คัดลอก API Key ไปยังไฟล์ .env ของคุณ


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# คำอธิบาย:
BASE_URL: นี่คือตัวแปรที่ใช้เก็บ URL พื้นฐานสำหรับจุดเชื่อมต่อของ SERP API ตัวแปรชื่อ BASE_URL เป็นชื่อที่ใช้ตามธรรมเนียมเพื่อบ่งบอกว่า URL นี้เป็นจุดเริ่มต้นสำหรับการส่งคำขอ API  
'https://serpapi.com/search':  

นี่คือตัวอักษร URL จริงที่ถูกกำหนดให้กับตัวแปร BASE_URL ซึ่งแสดงถึงจุดเชื่อมต่อสำหรับการทำคำค้นหาด้วย SERP API  

# วัตถุประสงค์:
วัตถุประสงค์ของบรรทัดนี้คือการกำหนดค่าคงที่ที่เก็บ URL พื้นฐานสำหรับ SERP API URL นี้จะถูกใช้เป็นจุดเริ่มต้นในการสร้างคำขอ API เพื่อดำเนินการค้นหา  

# การใช้งาน:
โดยการกำหนด URL พื้นฐานในตัวแปร คุณสามารถนำไปใช้ซ้ำได้ง่ายในโค้ดของคุณเมื่อใดก็ตามที่คุณต้องการส่งคำขอไปยัง SERP API วิธีนี้ทำให้โค้ดของคุณดูแลรักษาได้ง่ายขึ้นและลดความเสี่ยงของข้อผิดพลาดจากการเขียน URL ซ้ำในหลายที่ ตัวอย่างปัจจุบันคือ https://serpapi.com/search?engine=bing ซึ่งใช้ Bing search API สามารถเลือก API อื่นได้ที่ https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# คำอธิบาย:

นี่คือที่ที่โค้ดปลั๊กอินของคุณตั้งอยู่

การกำหนดคลาส: `class BookingPlugin`: กำหนดคลาสชื่อ BookingPlugin ที่มีเมธอดสำหรับการจองโรงแรมและเที่ยวบิน

เมธอดการจองโรงแรม:

- `@kernel_function(description="booking hotel")`: ตัวตกแต่งที่อธิบายฟังก์ชันว่าเป็นฟังก์ชันเคอร์เนลสำหรับการจองโรงแรม
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: กำหนดเมธอดสำหรับการจองโรงแรมพร้อมพารามิเตอร์ที่มีคำอธิบายและประเภทการคืนค่า

เมธอดนี้สร้างพจนานุกรมของพารามิเตอร์สำหรับคำขอจองโรงแรมและส่งคำขอ GET ไปยัง SERP API มันตรวจสอบสถานะการตอบกลับและคืนค่าคุณสมบัติของโรงแรมหากสำเร็จ หรือคืนค่า None หากคำขอล้มเหลว

เมธอดการจองเที่ยวบิน:

- `@kernel_function(description="booking flight")`: ตัวตกแต่งที่อธิบายฟังก์ชันว่าเป็นฟังก์ชันเคอร์เนลสำหรับการจองเที่ยวบิน
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: กำหนดเมธอดสำหรับการจองเที่ยวบินพร้อมพารามิเตอร์ที่มีคำอธิบายและประเภทการคืนค่า

เมธอดนี้สร้างพจนานุกรมของพารามิเตอร์สำหรับคำขอเที่ยวบินขาออกและขากลับ และส่งคำขอ GET ไปยัง SERP API มันตรวจสอบสถานะการตอบกลับและเพิ่มข้อมูลเที่ยวบินลงในสตริงผลลัพธ์หากสำเร็จ หรือพิมพ์ข้อความแสดงข้อผิดพลาดหากคำขอล้มเหลว เมธอดนี้คืนค่าสตริงผลลัพธ์ที่มีข้อมูลเที่ยวบิน


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# คำอธิบาย:
คำสั่งนำเข้า: นำเข้าโมดูลที่จำเป็นสำหรับข้อมูลรับรอง Azure, AI agent, เนื้อหาข้อความแชท, บทบาทผู้เขียน และตัวตกแต่งฟังก์ชัน kernel

ตัวจัดการบริบทแบบอะซิงโครนัส: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): ตั้งค่าตัวจัดการบริบทแบบอะซิงโครนัสเพื่อจัดการข้อมูลรับรอง Azure และสร้างไคลเอนต์ AI agent

ชื่อและคำแนะนำของ Agent:
- `AGENT_NAME = "BookingAgent"`: กำหนดชื่อของ agent
- `AGENT_INSTRUCTIONS = """..."""`: ให้คำแนะนำโดยละเอียดสำหรับ agent เกี่ยวกับวิธีจัดการคำขอการจอง

สร้างคำจำกัดความของ Agent: `agent_definition = await client.agents.create_agent(...)`: สร้างคำจำกัดความของ agent ด้วยโมเดล ชื่อ และคำแนะนำที่ระบุ

สร้าง AzureAI Agent: `agent = AzureAIAgent(...)`: สร้าง AzureAI agent โดยใช้ไคลเอนต์ คำจำกัดความของ agent และปลั๊กอินที่กำหนด

สร้าง Thread: `thread: AzureAIAgentThread | None = None`: สร้าง thread สำหรับ agent ไม่จำเป็นต้องสร้าง thread ก่อน - หากกำหนดค่าเป็น `None` thread ใหม่จะถูกสร้างขึ้นในระหว่างการเรียกครั้งแรกและส่งคืนเป็นส่วนหนึ่งของการตอบกลับ

ข้อมูลที่ผู้ใช้ป้อน: `user_inputs = ["..."]`: กำหนดรายการข้อมูลที่ผู้ใช้ป้อนสำหรับ agent เพื่อประมวลผล

ในบล็อกสุดท้าย ลบ thread และ agent เพื่อทำความสะอาดทรัพยากร


# การยืนยันตัวตน

คลาส `DefaultAzureCredential` เป็นส่วนหนึ่งของ Azure SDK สำหรับ Python ซึ่งให้วิธีการยืนยันตัวตนกับบริการของ Azure แบบเริ่มต้น โดยจะพยายามยืนยันตัวตนด้วยหลายวิธีตามลำดับที่กำหนดไว้ เช่น ตัวแปรสภาพแวดล้อม (environment variables), Managed Identity และ Azure CLI credentials

การทำงานแบบอะซิงโครนัส: โมดูล aio บ่งบอกว่าคลาส DefaultAzureCredential รองรับการทำงานแบบอะซิงโครนัส ซึ่งหมายความว่าคุณสามารถใช้ร่วมกับ asyncio เพื่อทำคำขอยืนยันตัวตนแบบไม่บล็อกการทำงานได้


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์มืออาชีพ เราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
